In [17]:
%%capture
!pip install requests  sentence-transformers transformers
!pip install git+https://github.com/huggingface/accelerate
!pip install langchain chromadb langchain
!pip install langchain_chroma langchain_community

In [1]:
import requests
from bs4 import BeautifulSoup

# Define your user agent string
headers = {'User-Agent': 'Your Name - your_email@example.com'}

def get_filings(ticker, filing_type="10-K"):
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={ticker}&type={filing_type}&dateb=&owner=exclude&count=100&output=xml"
    # print(url)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None
ticker="AAPL"
filings_xml = get_filings(ticker)
# print(filings_xml)

In [23]:
import requests, os
from bs4 import BeautifulSoup,NavigableString
import lxml
import re
import pandas as pd

print(bs4.__version__)
def parse_filings(filings_xml):
    """
    Gets the link from the "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=MSFT&type=10-K&dateb=&owner=exclude&count=100&output=xml"
    to Retrieve the target document from "https://www.sec.gov/Archives/edgar/data/789019/000095017023035122/0000950170-23-035122.txt"
    """
    soup = BeautifulSoup(filings_xml, 'xml')  # Parse as XML
    links = soup.find_all('filingHREF')
    base_url ="https://www.sec.gov"
    links_to_txt =[]
    for num, link in enumerate(links):
        # Want to fetch the past 10 years of 10-K documents
        if num>=10:
            break
            
        filing_page_url = link.text
        
        # Fetch the filing page content
        filing_page_content = requests.get(filing_page_url,headers=headers).text
        # Parse the filing page content
        filing_page_soup = BeautifulSoup(filing_page_content, 'html.parser')
        
        # Search for the target document link within the filing page
        document_link = filing_page_soup.find('a', href=lambda href: href and href.endswith('.txt'))
        # print(document_link)
        if document_link:
              links_to_txt.append((num, base_url + document_link.get('href')))
    if links_to_txt:
        return links_to_txt
   
    return None

document_url = parse_filings(filings_xml)
print(document_url)
print(f' before Sec base link : {document_url}')

# c_document_url=document_url.replace("ix?doc=/","")
# base_url ="https://www.sec.gov"


def Document_Content_Extractor(document_url):
    for num, document_link in document_url:    # document_url is a list of tuple i.e [(0, link_1) , (1, link_2)]
        raw_10k = requests.get(document_link, headers=headers).text
        # print(raw_10k[0:1400])

        # Regex to find <DOCUMENT> tags
        doc_start_pattern = re.compile(r'<DOCUMENT>')
        doc_end_pattern = re.compile(r'</DOCUMENT>')
        # Regex to find <TYPE> tag prceeding any characters, terminating at new line
        type_pattern = re.compile(r'<TYPE>[^\n]+')

        # Create 3 lists with the span idices for each regex

        ### There are many <Document> Tags in this text file, each as specific exhibit like 10-K, EX-10.17 etc
        ### First filter will give us document tag start <end> and document tag end's <start> 
        ### We will use this to later grab content in between these tags
        doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
        doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]

        ### Type filter is interesting, it looks for <TYPE> with Not flag as new line, ie terminare there, with + sign
        ### to look for any char afterwards until new line \n. This will give us <TYPE> followed Section Name like '10-K'
        ### Once we have have this, it returns String Array, below line will with find content after <TYPE> ie, '10-K' 
        ### as section names
        doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]

        document = {}

        # Create a loop to go through each section type and save only the 10-K section in the dictionary
        for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
            if doc_type == '10-K':
                document[doc_type] = raw_10k[doc_start:doc_end]


        # Write the regex
        regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(1A|1B|7A|7|8|9)\.{0,1})|(ITEM\s(1A|1B|7A|7|8|9))')

        # Use finditer to match the regex
        matches = regex.finditer(document['10-K'])

        # Write a for loop to print the matches
        # for match in matches:
        #     print(match)

        # Matches
        matches = regex.finditer(document['10-K'])

        # Create the dataframe
        test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

        test_df.columns = ['item', 'start', 'end']
        test_df['item'] = test_df.item.str.lower()

        # Display the dataframe

        test_df.replace('&#160;',' ',regex=True,inplace=True)
        test_df.replace('&nbsp;',' ',regex=True,inplace=True)
        test_df.replace(' ','',regex=True,inplace=True)
        test_df.replace('\.','',regex=True,inplace=True)
        test_df.replace('>','',regex=True,inplace=True)
        test_df.head()

        # Drop duplicates
        pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')

        # Set item as the dataframe index
        pos_dat.set_index('item', inplace=True)

        # Get Item 1a
        item_1a_raw = document['10-K'][pos_dat['start'].loc['item1a']:pos_dat['start'].loc['item1b']]

        # Get Item 7
        item_7_raw = document['10-K'][pos_dat['start'].loc['item7']:pos_dat['start'].loc['item7a']]

        # Get Item 7a
        item_7a_raw = document['10-K'][pos_dat['start'].loc['item7a']:pos_dat['start'].loc['item8']]

        item_8_raw = document['10-K'][pos_dat['start'].loc['item8']:pos_dat['end'].loc['item9']]

    def Extractor(item_num_raw):
        item_num_content = BeautifulSoup(item_num_raw, 'lxml')
        return item_num_content.get_text("\n")
    
    for i in [item_1a_raw, item_7_raw, item_7a_raw, item_8_raw]:
        
    
# print(item_8_content.get_text("\n\n")[0:15000])
# exit()
# def get_text_from_innermost_tags(tag):
#     if isinstance(tag, NavigableString):
#         return tag.strip()
#     else:
#         # Check if this tag contains other tags
#         if tag.find() is not None:
#             # This tag contains other tags, so ignore it
#             return ''
#         else:
#             # This tag does not contain other tags, so extract its text
#             return tag.get_text(separator='\n', strip=True) +'\n'
            
# soup = BeautifulSoup(html_content, 'html.parser')
# start_tag = soup.find(lambda tag: tag.name == "span" and "UNITED STATES" in tag.get_text())
# # print(start_tag)
# if start_tag:
#     # Extract the HTML from the starting tag onwards
#     relevant_content = start_tag.get_text(separator='\n', strip= True)
#     for element in start_tag.find_all_next():
#         # print(element)
#         relevant_content += get_text_from_innermost_tags(element) 
# else:
#     print("Starting point not found in the document")
#     relevant_content = ""

# # print(relevant_content)

4.12.3
[(0, 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/0000320193-23-000106.txt'), (1, 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/0000320193-22-000108.txt'), (2, 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000105/0000320193-21-000105.txt'), (3, 'https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/0000320193-20-000096.txt'), (4, 'https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/0000320193-19-000119.txt'), (5, 'https://www.sec.gov/Archives/edgar/data/320193/000032019318000145/0000320193-18-000145.txt'), (6, 'https://www.sec.gov/Archives/edgar/data/320193/000032019317000070/0000320193-17-000070.txt'), (7, 'https://www.sec.gov/Archives/edgar/data/320193/000162828016020309/0001628280-16-020309.txt'), (8, 'https://www.sec.gov/Archives/edgar/data/320193/000119312515356351/0001193125-15-356351.txt'), (9, 'https://www.sec.gov/Archives/edgar/data/320193/000119312514383437/0001193125-14-383437.txt')]


AttributeError: module 'os' has no attribute 'exit'

In [15]:
import bs4
print(bs4.__version__)

4.12.3


In [16]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
with open('output.txt', 'w') as file:
    file.write(relevant_content)

loader = TextLoader('output.txt')
documents= loader.load()

text_splitter = CharacterTextSplitter(chunk_size =1000, chunk_overlap =200)
docs = text_splitter.split_documents(documents)

model_name = "all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_function = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
db = Chroma.from_documents(docs,embedding=embedding_function)

query = "what is the net Income for the year 2023"
docs= db.similarity_search(query)

print(docs[0].page_content)



NameError: name 'relevant_content' is not defined

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, accelerate

device ="cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/OpenHermes-2.5-Mistral-7B-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
query= "What is the Debt ratio of this company?"
retriever = db.as_retriever(k=1)
relevant_chunks = retriever.invoke(query)
chunk_texts = [doc.page_content for doc in relevant_chunks]
combined_input = "Userquery :" + query + "\n\n" + "\n Context:".join(chunk_texts)

input_ids = tokenizer(combined_input, return_tensors="pt").input_ids.to(device)

output = model.generate(input_ids, max_new_tokens=200, num_return_sequences=1)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Userquery :What is the Debt ratio of this company?

Amount


Weighted
Average Life

Year Ended June 30,

2023

2022

Technology-based

$
522

7
years


$
2,611

4
years

Customer-related


0

0
years

2,837

9
years

Marketing-related


7

5
years

233

4
years

Contract-based


12

3
years

0

0
years


Total

$
541

6
years


$

5,681

7
years

Intangible assets amortization expense was $
2.5
billion, $
2.0
billion, and $
1.6
billion for fiscal years 2023, 2022, and 2021, respectively.

80

PART II
Item 8

The following table outlines the estimated future amortization expense related to intangible assets held as of June 30, 2023:


(In millions)


Year Ending June 30,


2024

$
2,363

2025


1,881

2026


1,381

2027


929

2028


652

Thereafter


2,160


Total

$
9,366

NOTE 11 — DEBT

The components of debt were as follows:

(In millions, issuance by calendar year)

Maturities
(calendar year)

Stated Interest
Rate


Effective Interest
Rate

June 30,
2023


June 30,
2022


2009
iss

In [ ]:
import torch
import gc

# Assume you have some model and data loaded
# model = ...
# inputs = ...

# Perform operations
# output = model(inputs)

# Now clear GPU memory

gc.collect()  # run garbage collection
torch.cuda.empty_cache()  # clear the PyTorch cache



In [ ]:
!nvidia-smi

Sat Jun  8 23:11:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              35W /  70W |   7439MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import re

def detect_and_convert_to_markdown(text):
    lines = text.strip().split("\n")
    rows = []
    current_row = []
    max_columns = 0

    # Helper function to add row if it's complete
    def add_row_if_complete():
        nonlocal max_columns
        if current_row:
            rows.append(current_row[:])
            if len(current_row) > max_columns:
                max_columns = len(current_row)
            current_row.clear()

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Split the line by spaces, tabs, or other delimiters to detect columns
        columns = re.split(r'\s{2,}|\t', line)
        if len(columns) > 1:
            add_row_if_complete()
            current_row.extend(columns)
            add_row_if_complete()
        else:
            current_row.append(line)
            add_row_if_complete()

    # Handle the last row if any
    add_row_if_complete()

    # Construct the markdown table
    if rows:
        headers = ["Column " + str(i + 1) for i in range(max_columns)]
        markdown_table = f"| {' | '.join(headers)} |\n"
        markdown_table += f"|{'|'.join(['-' * (len(header) + 2) for header in headers])}|\n"

        for row in rows:
            markdown_table += f"| {' | '.join(row + [''] * (max_columns - len(row)))} |\n"

        return markdown_table
    else:
        return "No table detected in the text."

# Sample input text
text = generated_text

# Generate the markdown table
markdown_table = detect_and_convert_to_markdown(text)
print(markdown_table)

# Optionally, save to a markdown file
with open("output_table.md", "w") as file:
    file.write(markdown_table)



| Column 1 |
|----------|
| Userquery :What is the Debt ratio of this company? |
| Amount |
| Weighted |
| Average Life |
| Year Ended June 30, |
| 2023 |
| 2022 |
| Technology-based |
| $ |
| 522 |
| 7 |
| years |
| $ |
| 2,611 |
| 4 |
| years |
| Customer-related |
| 0 |
| 0 |
| years |
| 2,837 |
| 9 |
| years |
| Marketing-related |
| 7 |
| 5 |
| years |
| 233 |
| 4 |
| years |
| Contract-based |
| 12 |
| 3 |
| years |
| 0 |
| 0 |
| years |
| Total |
| $ |
| 541 |
| 6 |
| years |
| $ |
| 5,681 |
| 7 |
| years |
| Intangible assets amortization expense was $ |
| 2.5 |
| billion, $ |
| 2.0 |
| billion, and $ |
| 1.6 |
| billion for fiscal years 2023, 2022, and 2021, respectively. |
| 80 |
| PART II |
| Item 8 |
| The following table outlines the estimated future amortization expense related to intangible assets held as of June 30, 2023: |
| (In millions) |
| Year Ending June 30, |
| 2024 |
| $ |
| 2,363 |
| 2025 |
| 1,881 |
| 2026 |
| 1,381 |
| 2027 |
| 929 |
| 2028 |
| 652 |
| Therea